# Modelo de regresión simple

Sea $ϵ_1,..., ϵ_n $ iid N~[0; $σ^2$]. Considere que cada $ϵ_1$ puede ser escrito como una función de datos observados para $x_i$ y $y_i$. Es decir, $ϵ_1$ = $y_i-α−βx_ 
i$

### 1. Escriba la funcion log-likelihood.

La función de densidad condicional para una observación es:

$$
f(y_i \mid x_i; \alpha, \beta, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(y_i - \alpha - \beta x_i)^2}{2\sigma^2} \right)
$$

Entonces la funcion de log-likelihood es:

$$
\log L(\alpha, \beta, \sigma^2) = \sum_{i=1}^n \log f(y_i \mid x_i; \alpha, \beta, \sigma^2)
$$

$$
= -\frac{n}{2} \log(2\pi) - \frac{n}{2} \log(\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - \alpha - \beta x_i)^2
$$

2. Encuentre el estimador de maxima verosimilitud para α, β, $σ^2$

Primero maximizamos la funcion log-likelihood con respecto a α para encontrar α^:
$$
\frac{\partial \log L}{\partial \alpha} = \frac{1}{\sigma^2} \sum_{i=1}^n (y_i - \alpha - \beta x_i)=0
$$
Y encontramos que α^ es: 
$$
\hat{\alpha} =  \frac{1}{n} \sum_{i=1}^n (y_i  - \hat{\beta} x_i)=  \bar{y} - \hat{\beta} \bar{x}
$$


Luego sustituimos el α^ y maximizamos con respecto a β para encontrar β^:
$$
\frac{dlog L}{d\beta} = \frac{-1}{ \sigma^2} \sum_{i=1}^n \left[ (y_i - \bar{y}) - \beta(x_i - \bar{x}) \right](x_i - \bar{x}) = 0
$$
Despejamos β y encontramos β^:
$$
\hat{\beta} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2}
$$


Finalmente derivamos con respecto a $σ^2$
$$

\frac{dlog L}{d\sigma^2}= -\frac{n}{2 \sigma^2} + \frac{1}{2 \sigma^4} \sum_{i=1}^n (y_i - \alpha - \beta x_i)^{2}
$$
y obtenemos que:
$$
\hat{\sigma^2}= \frac{1}{n} \sum{i=1}^n (y_i - \hat{\alpha} - \hat{\beta} x_i)^{2}
$$
dado que:
$$
RSS= \sum{i=1}^n (y_i - \hat{\alpha} - \hat{\beta} x_i)^{2}
$$
Entonces:
$$
\hat{\sigma^2}= \frac{RSS}{n} 

### 3. Encuentre el limite inferior Cramer-Rao.